<a href="https://colab.research.google.com/github/sean-halpin/ml_tweepy_proj/blob/main/svm_text_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir("/content/drive/My Drive/Colab Notebooks")

In [ ]:
!pip install vaderSentiment
!pip install spellchecker
!pip install pyspellchecker
!pip install plot_keras_history

In [ ]:
import numpy as np
import pandas as pd 
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import *
from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
stop_words = set(stopwords.words('english'))
def no_stopwords(text):
  tokenwords = word_tokenize(text) 
  result = [w for w in tokenwords if not w in stop_words] 
  result = [] 
  for w in tokenwords: 
      if w not in stop_words: 
          result.append(w)
  return " ".join(result)

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])

In [ ]:
output= string.punctuation
print('list of punctuations:', output)

In [ ]:
def prepare_text_df(df):
  df['tweet'] = df['tweet'].str.replace('http\S+|www.\S+|@.\S+|&amp;.\S+|<.*?>', '', case=False)
  df['tweet'] = df['tweet'].str.lower()
  df['tweet'] = df['tweet'].str.strip()
  df['tweet'] = df['tweet'].str.translate(str.maketrans('', '', output))
  df['lemmatized_tweet'] = df['tweet'].apply(lemmatize_text).apply(no_stopwords)
  df = df.drop_duplicates('lemmatized_tweet', keep='last')
  return df

In [ ]:
def random_under_sample(X,y):
  rus = RandomUnderSampler(random_state=0)
  X_resampled, y_resampled = rus.fit_resample(X,y)
  print(sorted(Counter(y_resampled).items()))
  return X_resampled.flatten(), y_resampled.flatten()

In [ ]:
def load_data_musk():
  df = pd.read_csv("tweets_annotated.elon_musk.1651256232.csv")
  df = prepare_text_df(df)
  X = df.lemmatized_tweet.values.reshape(-1,1)
  y = df.sentiment.values.reshape(-1,1)
  X_resampled, y_resampled = random_under_sample(X,y)
  df_resampled = pd.DataFrame ({ 'lemmatized_tweet': X_resampled, 'sentiment': y_resampled})
  return df_resampled

In [ ]:
df_resampled = load_data_musk()

In [ ]:
df_resampled.head()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_resampled.lemmatized_tweet, df_resampled.sentiment, test_size=0.2, random_state=32)

# Text Classifier

In [ ]:
text_clf = Pipeline([
    ('vect', CountVectorizer(max_features=1000)),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=3, tol=None)),
])

text_clf.fit(X_train, y_train)

predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import seaborn as sns
def clf_metrics(y_test, predicted):
  print(metrics.classification_report(y_test, predicted))
  c_matrix = metrics.confusion_matrix(y_test, predicted)
  ax = sns.heatmap(c_matrix, annot=True,      
                    xticklabels=['negative','neutral','positive'],                
                    yticklabels=['negative','neutral','positive'],                
                  cbar=True, cmap='Blues', fmt='g')
  ax.set_xlabel("Prediction")
  ax.set_ylabel("Actual")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve


def plot_learning_curve(
    estimator,
    title,
    X,
    y,
    axes=None,
    ylim=None,
    cv=None,
    n_jobs=None,
    train_sizes=np.linspace(0.1, 1.0, 5),
):
    if axes is None:
        _, (ax) = plt.subplots(1, 1, figsize=(20, 5))

    ax.set_title(title)
    if ylim is not None:
        ax.set_ylim(*ylim)
    ax.set_xlabel("Training examples")
    ax.set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(
        estimator,
        X,
        y,
        cv=cv,
        n_jobs=n_jobs,
        train_sizes=train_sizes,
        return_times=True,
    )
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    ax.grid()
    ax.fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="r",
    )
    ax.fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
    ax.plot(
        train_sizes, train_scores_mean, "o-", color="r", label="Training score"
    )
    ax.plot(
        train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score"
    )
    ax.legend(loc="best")
    plt.show()
    return plt

In [ ]:
clf_metrics(y_test, predicted)

In [ ]:
plot_learning_curve(
    text_clf, "LC ", X=X_train, y=y_train,ylim=(0.1, 1.01), cv=5, n_jobs=-1
)

# SVM Classification Performance against another topic

In [ ]:
def load_data_formula_one():
  df_f1 = pd.read_csv("tweets_annotated.mercedes_f1.1651259463.csv")
  df_f1 = prepare_text_df(df_f1)
  X_f1 = df_f1.lemmatized_tweet.values.reshape(-1,1)
  y_f1 = df_f1.sentiment.values.reshape(-1,1)
  X_f1_resampled, y_f1_resampled = random_under_sample(X_f1,y_f1)
  return pd.DataFrame ({ 'lemmatized_tweet': X_f1_resampled, 'sentiment': y_f1_resampled})

In [ ]:
df_f1_resampled = load_data_formula_one()

In [ ]:
predicted_f1 = text_clf.predict(df_f1_resampled.lemmatized_tweet)

In [ ]:
clf_metrics(df_f1_resampled.sentiment, predicted_f1)

# Vader

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
analyzer = SentimentIntensityAnalyzer()
def vader_sentiment(sentence):
  vs = analyzer.polarity_scores(sentence)
  comp = vs['compound']
  if comp > 0.05:
    return 'positive'
  elif comp < -0.05:
    return 'negative'
  else:
    return 'neutral'

In [ ]:
vader_predicted = X_test.apply(vader_sentiment)

In [ ]:
clf_metrics(y_test, vader_predicted)

In [ ]:
len(X_test[(y_test == "neutral") & (vader_predicted == "positive")])

# Deep Neural Network Classifier

In [ ]:
import numpy as np
import pandas as pd
import re
import gensim
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
df_resampled = load_data_musk()

In [ ]:
def labels_categorical(data):
  labels = np.array(data)
  y = []
  for i in range(len(labels)):
      if labels[i] == 'neutral':
          y.append(0)
      if labels[i] == 'negative':
          y.append(1)
      if labels[i] == 'positive':
          y.append(2)
  y = np.array(y)
  labels = tf.keras.utils.to_categorical(y, 3, dtype="float32")
  del y
  return labels

In [ ]:
labels = labels_categorical(df_resampled.sentiment)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers

max_words = 500
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df_resampled.lemmatized_tweet)
sequences = tokenizer.texts_to_sequences(df_resampled.lemmatized_tweet)
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tweets, labels, test_size=0.3, random_state=32)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.layers import Embedding
embedding_layer = Embedding(128, 16)

model1 = Sequential()
model1.add(layers.Embedding(max_words, 12))
model1.add(layers.LSTM(12,dropout=0.4, return_sequences=False))
model1.add(layers.Dense(3,activation='softmax'))

model1.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model1.summary()

In [ ]:
#Implementing model checkpoins to save the best metric and do not lose it on training.
checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model1.fit(X_train, y_train, epochs=20,validation_data=(X_test, y_test),callbacks=[checkpoint1])

In [ ]:
import keras

best_model = keras.models.load_model("best_model1.hdf5")

In [ ]:
test_loss, test_acc = best_model.evaluate(X_test, y_test, verbose=2)
print('Model accuracy: ',test_acc)

In [ ]:
predictions = best_model.predict(X_test)

In [ ]:
clf_metrics(y_test.argmax(axis=1), np.around(predictions, decimals=0).argmax(axis=1))

In [ ]:
from plot_keras_history import show_history, plot_history
import matplotlib.pyplot as plt

show_history(history)
plot_history(history, path="standard.png")
plt.close()

# Report - Model Comparison SVM vs. Vader

# Visualisations

In [ ]:
df = pd.read_csv("tweets_annotated.mercedes_f1.1651259463.csv")

In [ ]:
df.head()

In [ ]:
df['created_at'] = pd.to_datetime(df['created_at'])

In [ ]:
tweet_df_5min = df.groupby(pd.Grouper(key='created_at', freq='30Min', convention='start')).size()
tweet_df_5min.plot(figsize=(18,6))
plt.ylabel('30 Minute Tweet Count')
plt.title('Formula One Tweet Freq. Count')
plt.grid(True)

In [ ]:
from wordcloud import WordCloud

In [ ]:
def prep_text_for_wordcloud(df):
  df['cleaned_tweet'] = df['tweet'].str.replace('https\S+|http\S+|www.\S+|@.\S+|&amp;.\S+|<.*?>', '', case=False)
  df['cleaned_tweet'] = df['cleaned_tweet'].str.lower()
  df['cleaned_tweet'] = df['cleaned_tweet'].str.strip()
  df['normalized_tweet'] = df['cleaned_tweet'].str.translate(str.maketrans('', '', output))
  df['normalized_tweet'] = df['normalized_tweet'].apply(no_stopwords)
  return df

In [ ]:
wordcloud_df = prep_text_for_wordcloud(df)

In [ ]:
def create_wordcloud(wc_title, series):
  wordcloud = WordCloud(width=800, height=400, margin=2).generate(' '.join(series))
  plt.figure(figsize=(10,10))
  plt.title(wc_title)
  plt.axis("off")
  plt.imshow(wordcloud)

In [ ]:
hashtag_counts = df.tweet.str.extractall(r'(\#\w+)')[0].value_counts()

In [ ]:
top_hashtags = hashtag_counts[:10]

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
keys = top_hashtags.keys().to_list()
values = list(top_hashtags.array)
ax.bar(keys,values)
plt.title("Most Common Hashtags")
plt.xticks(rotation=90)
plt.xlabel("Hashtag")
plt.ylabel("Count")
plt.show()

In [ ]:
positive_tweets = wordcloud_df[wordcloud_df["sentiment"] == "positive"]['normalized_tweet']
neutral_tweets = wordcloud_df[wordcloud_df["sentiment"] == "neutral"]['normalized_tweet']
negative_tweets = wordcloud_df[wordcloud_df["sentiment"] == "negative"]['normalized_tweet']
tweets_with_likes = wordcloud_df[wordcloud_df["like_count"] > 10]['normalized_tweet']
tweets_with_retweets = wordcloud_df[wordcloud_df["retweet_count"] > 5]['normalized_tweet']

In [ ]:
len(tweets_with_retweets)

In [ ]:
create_wordcloud("All Formula One Tweets - Wordcloud", wordcloud_df['normalized_tweet'])
create_wordcloud("Positive Formula One Tweets - Wordcloud",  positive_tweets)
create_wordcloud("Neutral Formula One Tweets - Wordcloud",  neutral_tweets)
create_wordcloud("Negative Formula One Tweets - Wordcloud",  negative_tweets)
create_wordcloud("Liked Formula One Tweets - Wordcloud",  tweets_with_likes)
create_wordcloud("Retweeted Formula One Tweets - Wordcloud",  tweets_with_retweets)

In [ ]:
import requests

from shapely.geometry import mapping, shape
from shapely.prepared import prep
from shapely.geometry import Point


data = requests.get(
    "https://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson").json()

countries = {}
for feature in data["features"]:
    geom = feature["geometry"]
    country = feature["properties"]["ADMIN"]
    countries[country] = prep(shape(geom))

def get_country(lon, lat):
    point = Point(lon, lat)
    for country, geom in countries.items():
        if geom.contains(point):
            return country

    return "unknown"

In [ ]:
from wordcloud import WordCloud

def create_wordcloud(series):
  wordcloud = WordCloud(width=800, height=400, margin=2).generate(' '.join(series))
  return wordcloud.to_image()

In [ ]:
import string
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))


def no_stopwords(text):
    tokenwords = word_tokenize(text)
    result = [w for w in tokenwords if not w in stop_words]
    result = []
    for w in tokenwords:
        if w not in stop_words:
            result.append(w)
    return " ".join(result)


def prep_text(df):
    df['sentiment_category'] = df['sentiment'].astype('category')
    df['sentiment_numeric'] = pd.factorize(df['sentiment_category'])[0]
    df['cleaned_tweet'] = df['tweet'].str.replace(
        'https\S+|http\S+|www.\S+|@.\S+|&amp;.\S+|<.*?>', '', case=False)
    df['cleaned_tweet'] = df['cleaned_tweet'].str.lower()
    df['cleaned_tweet'] = df['cleaned_tweet'].str.strip()
    df['normalized_tweet'] = df['cleaned_tweet'].str.translate(
        str.maketrans('', '', string.punctuation))
    df['normalized_tweet'] = df['normalized_tweet'].apply(no_stopwords)
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['country'] = df.apply(
        lambda x: get_country(x['long'], x['lat']), axis=1)
    return df


In [ ]:
!pip install -U kaleido

In [ ]:
import plotly.express as px
# df = pd.read_csv("tweets_annotated.1650575029.formulaone.csv")
df = pd.read_csv("tweets_annotated.elon_musk.1651256232.csv")
df = prep_text(df)

# Line - Tweets Over Time Buckets
tweet_df_buckets = df.groupby(pd.Grouper(
    key='created_at', freq='5Min', convention='start')).size()
df_t = pd.DataFrame(list(tweet_df_buckets.items()),
                    columns=['created_at', 'counts'])
fig_buckets = px.line(
    df_t,
    x="created_at",
    y="counts",
    title="Tweet Counts in 5 minute buckets",
    labels={
        'created_at': 'Time',
        'counts': 'Count'
    }
)

# Lines - Tweet Sentiment Counts in 60 minute Time Buckets
tweet_sentiment_time_buckets = df.groupby([
    'sentiment',
    pd.Grouper(
        key='created_at', freq='5Min', convention='start')
]).size()
df_t_s = tweet_sentiment_time_buckets.reset_index(name='counts')

fig_tweet_sentiment_time_buckets = px.line(
    df_t_s,
    line_group="sentiment",
    color="sentiment",
    x="created_at",
    y="counts",
    title="Tweet Sentiment Counts in 5 minute Time Buckets",
    labels={
        'created_at': 'Time',
        'sentiment': 'Sentiment',
        'counts': 'Count'
    }
)

# Pie - Sentiment
df_sent_counts = df['sentiment'].value_counts().reset_index(name='counts')
fig_pie_overall_sent = px.pie(
    df_sent_counts,
    values='counts',
    names='index',
    title='Overall Sentiment Pie Chart',
    hole=0.2,
    labels={
        'index': 'Sentiment',
        'counts': 'Count'
    }
)

# Chloropleth - Median Tweet Sentiment by Country
tweet_most_common_sentiment_by_country = df.groupby(
    'country')['sentiment_numeric'].mean().reset_index(name='sentiment_mean')
fig_chloro_average_sentiment = px.choropleth(
    tweet_most_common_sentiment_by_country,
    locations="country",
    locationmode="country names",
    projection="natural earth",
    hover_name="country",
    color="sentiment_mean",
    color_continuous_scale='aggrnyl',
    color_continuous_midpoint=0.4,
    title="Sentiment Positivity by Country",
    labels={
        'country': 'Country',
        'sentiment_mean': 'Sentiment Positivity',
        'counts': 'Count'
    },
    width=1500,
    height=800
)

# Sunburst sentiments by platform
tweet_sentiments_by_source = df.groupby(['source', 'sentiment'])['sentiment'].count().reset_index(name='count')
top_platforms = df['source'].value_counts()[:5].to_frame().reset_index()['index'].to_list()
tweet_sentiments_by_platform_top = tweet_sentiments_by_source[tweet_sentiments_by_source['source'].isin(top_platforms)]
fig_sunb = px.sunburst(
    tweet_sentiments_by_platform_top,
    path=["source", "sentiment"],
    values='count',
    height=1000,
    title='Sentiment Counts per Platform',
    color='sentiment'
)

# Treemap
fig_treemap = px.treemap(
    tweet_sentiments_by_platform_top,
    path=["source", "sentiment"],
    height=1000,
    title='Sentiment Counts per Platform',
    color='sentiment',
    values='count'
)

# Geo - Tweet Counts
df_country_counts = df['country'].value_counts().reset_index(name='counts')
# print(df_country_counts)
fig_geo_tweet_counts = px.scatter_geo(
    df_country_counts,
    locations="index",
    locationmode="country names",
    size="counts",
    projection="natural earth",
    hover_name="index",
    color="index",
    title="Relative Tweet Count per Country",
    labels={
        'index': 'Country',
        'counts': 'Count'
    },
    width=1500,
    height=800
)

# Most popular hashtags
hashtag_counts = df.tweet.str.extractall(
    r'(\#\w+)')[0].value_counts().reset_index(name='counts')
fig_popular_hashtags = px.bar(
    hashtag_counts[hashtag_counts.counts > 10][:10],
    x='index',
    y='counts',
    color='counts',
    title="Most Popular Hashtags",
    labels={
        'index': 'Hashtag',
        'counts': 'Count'
    }
)

# Wordcloud
fig_worldcloud = px.imshow(
    create_wordcloud(df['normalized_tweet']),
    title="Wordcloud"
)

# Bar - Mean Tweet Sentiment by Platform
most_frequent_platforms = df['source'].value_counts()[:10].index.tolist()
tweet_most_common_sentiment_by_platform = df[df['source'].isin(most_frequent_platforms)].groupby(
    'source')['sentiment_numeric'].mean().reset_index(name='sentiment_mean')
fig_tweet_most_common_sentiment_by_platform = px.bar(
    tweet_most_common_sentiment_by_platform.sort_values(by=['sentiment_mean']),
    x='source',
    y='sentiment_mean',
    color='source',
    color_continuous_scale='thermal',
    title="Average Sentiment Positivity by Platform",
    labels={
        'source': 'Platform',
        'sentiment_mean': 'Sentiment Positivity',
    }
)

# Mabox Density
fig_rel = px.density_mapbox(
    df, lat='lat', lon='long', z='reliability', radius=10,
    center=dict(lat=0, lon=180), zoom=0,
    mapbox_style="stamen-terrain"
)

# Chloropleth - Median Tweet Reliability by Country
tweet_avg_reliability_by_country = df.groupby(
    'country')['reliability'].mean().reset_index(name='reliability_mean')
fig_chloro_average_reliability = px.choropleth(
    tweet_avg_reliability_by_country,
    locations="country",
    locationmode="country names",
    projection="natural earth",
    hover_name="country",
    color="reliability_mean",
    color_continuous_scale='thermal',
    color_continuous_midpoint=0.7,
    title="Mean Reliability by Country",
    labels={
        'country': 'Country',
        'reliability_mean': 'Reliability',
        'counts': 'Count'
    },
    width=1500,
    height=800
)

In [169]:
fig_buckets.show(renderer="colab")